In [1]:
import cv2
import itertools as it
import math
import numpy as np
import os
import pickle
import random
from collections import Counter
from utilities import *

In [5]:
file_titles = ('2020-04-11-telshin-3', '2020-04-13-mixwell-1', '2020-04-14-myth-0')
top, left, bottom, right = 39, 678, 242, 725
width, height = right - left, 11
color = np.array([0.52949995, 0.6719118, 0.70135915], dtype=np.float32)
luv = np.array([70.05418, 6.4813266, 27.519352], dtype=np.float32)
composite = cv2.cvtColor(np.broadcast_to(luv, (1, 1, 3)), cv2.COLOR_LUV2BGR).reshape(-1)
color_template, luv_template, composite_template = (np.broadcast_to(i, (height, width, 3)) for i in (color, luv, composite))
color, luv, composite

(array([0.52949995, 0.6719118 , 0.70135915], dtype=float32),
 array([70.05418  ,  6.4813266, 27.519352 ], dtype=float32),
 array([0.5308992, 0.6740422, 0.7026945], dtype=float32))

In [ ]:
def fn(file_title):
    input_file_path = fr'D:\Valorant\Videos\{file_title}.mp4'
    output_file_path = fr'D:\Valorant\Killed\Pickles\{file_title}.pickle'
    video = cv2.VideoCapture(input_file_path)
    with open(output_file_path, 'wb') as fout:
        while True:
            _, frame = video.read()
            if frame is None:
                break
            image = frame[top:bottom, left:right].astype(np.float32) / 255
            pickle.dump(image, fout)
    video.release()
for s in file_titles:
    fn(s)

In [3]:
def fn(file_path):
    with open(file_path) as fin:
        return fin.readline().rstrip()
g = (fr'D:\Valorant\Killed\Labels\{s}.txt' for s in file_titles)
g = map(fn, g)
g = it.chain.from_iterable(g)
labels = ''.join(g)
Counter(labels)

Counter({'n': 84497, 'y': 30976, 'z': 1})

In [4]:
def fn(g, n, k):
    for v in g:
        if random.randrange(n) < k:
            yield v
            k -= 1
            if k == 0:
                break
        n -= 1
g = (fr'D:\Valorant\Killed\Pickles\{s}.pickle' for s in file_titles)
g = map(pickle_iter, g)
g = it.chain.from_iterable(g)
g = zip(g, labels)
g = ((i, l) for i, l in g if l != 'z')
data = list(fn(g, len(labels), 20000))
Counter(l for i, l in data)

Counter({'n': 14604, 'y': 5396})

In [10]:
def fn(image, label):
    match = cv2.matchTemplate(image, color_template, cv2.TM_SQDIFF)
    result = match.min() < 46
    return label, result
[Counter(it.starmap(fn, data))]

[Counter({('n', False): 9613,
          ('n', True): 4991,
          ('y', True): 5375,
          ('y', False): 21})]

In [6]:
def fn(image, label):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2LUV)
    match = cv2.matchTemplate(image, luv_template, cv2.TM_SQDIFF)
    result = match.min() < 500000 or len(match[match < 2 * match.min()]) < 60
    return label, result
[Counter(it.starmap(fn, data))]

[Counter({('n', False): 7357,
          ('n', True): 7247,
          ('y', True): 5298,
          ('y', False): 98})]

In [8]:
def fn(image, label):
    match = cv2.matchTemplate(image, composite_template, cv2.TM_SQDIFF)
    result = match.min() < 42 or len(match[match < 2 * match.min()]) < 30
    return label, result
[Counter(it.starmap(fn, data))]

[Counter({('n', False): 9042,
          ('n', True): 5562,
          ('y', True): 5374,
          ('y', False): 22})]